In [1]:
import glob
import pandas as pd
import numpy as np
import sys
import operator
import numpy as np
from scipy import stats
import requests
from IPython.display import HTML
import matplotlib.pylab as plt
import warnings
import geopandas as gpd
from fiona.crs import from_epsg

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
# pluto = pd.read_csv("MN.csv")
# pluto.head()

In [3]:
# importing pluto sas area shapefile
poi_pluto = gpd.GeoDataFrame(gpd.read_file("area_of_interest.shp"))
poi_pluto.head()


,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,...,APPDate,PLUTOMapID,Version,MAPPLUTO_F,SHAPE_Leng,SHAPE_Area,FID_1,distance_m,nearest_st,geometry
0,MN,1440,49,108,118,5000,02,5,10065,L016,...,05/07/2009,1,16V2,0,618.608412,21097.198093,0,524.991986,0,"POLYGON ((995131.005400002 217889.4583999962, ..."
1,MN,1422,7501,108,118,3000,02,4,10065,L016,...,05/01/2006,1,16V2,7,310.861563,5401.336650,1,391.962637,0,"POLYGON ((994921.264200002 218484.6298000067, ..."
2,MN,1422,36,108,118,3000,02,4,10065,L016,...,None,1,16V2,0,236.647242,1791.806600,2,363.777781,0,"POLYGON ((994969.8884000033 218572.5766000003,..."
3,MN,1422,2,108,118,3000,02,4,10065,L016,...,None,1,16V2,0,354.522266,7696.616852,3,419.980016,0,"POLYGON ((994713.3367999941 218599.4177999943,..."
4,MN,1424,43,108,126,4000,02,4,10021,L016,...,None,1,16V2,0,270.246724,2999.465059,4,247.816025,0,"POLYGON ((995064.7611999959 219118.5095999986,..."


In [4]:
poi_pluto.columns

Index([   u'Borough',      u'Block',        u'Lot',         u'CD',
           u'CT2010',     u'CB2010', u'SchoolDist',    u'Council',
          u'ZipCode',   u'FireComp', u'PolicePrct', u'HealthArea',
        u'SanitBoro', u'SanitDistr',   u'SanitSub',    u'Address',
        u'ZoneDist1',  u'ZoneDist2',  u'ZoneDist3',  u'ZoneDist4',
         u'Overlay1',   u'Overlay2',    u'SPDist1',    u'SPDist2',
          u'SPDist3',  u'LtdHeight',  u'SplitZone',  u'BldgClass',
          u'LandUse',  u'Easements',  u'OwnerType',  u'OwnerName',
          u'LotArea',   u'BldgArea',    u'ComArea',    u'ResArea',
       u'OfficeArea', u'RetailArea', u'GarageArea',  u'StrgeArea',
       u'FactryArea',  u'OtherArea', u'AreaSource',   u'NumBldgs',
        u'NumFloors',   u'UnitsRes', u'UnitsTotal',   u'LotFront',
         u'LotDepth',  u'BldgFront',  u'BldgDepth',        u'Ext',
         u'ProxCode', u'IrrLotCode',    u'LotType',   u'BsmtCode',
       u'AssessLand',  u'AssessTot', u'ExemptLand',  u'ExemptT

In [5]:
# importing rolling sales data
sales = pd.read_csv('rollingsales_final.CSV')
sales.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,BOROUGHBLOCKLOT
0,1.0,ALPHABET CITY,03 THREE FAMILY DWELLINGS,1,376.0,24.0,,C0,264 EAST 7TH STREET,,...,0,3,"\t2,059","\t3,696",1900,1.0,C0,"\t7,738,700",12/22/16,137624.0
1,1.0,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,375.0,28.0,,C4,738 EAST 6TH STREET,,...,0,11,"\t1,750","\t6,500",1900,2.0,C4,"\t3,750,000",4/3/17,137528.0
2,1.0,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,385.0,36.0,,C7,27 AVENUE C,,...,1,25,"\t2,650","\t9,960",1910,2.0,C7,"\t5,235,000",7/11/17,138536.0
3,1.0,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392.0,5.0,,C2,151 AVENUE B,,...,0,5,"\t2,139","\t4,416",1900,2.0,C2,\t-,6/16/17,13925.0
4,1.0,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392.0,6.0,,C2,153 AVENUE B,,...,0,5,"\t1,633","\t6,440",1900,2.0,C2,"\t6,625,000",7/19/17,13926.0


In [6]:
sales.columns

Index([u'BOROUGH', u'NEIGHBORHOOD', u'BUILDING CLASS CATEGORY',
       u'TAX CLASS AT PRESENT', u'BLOCK', u'LOT', u'EASE-MENT',
       u'BUILDING CLASS AT PRESENT', u'ADDRESS', u'APARTMENT NUMBER',
       u'ZIP CODE', u'RESIDENTIAL UNITS', u'COMMERCIAL UNITS', u'TOTAL UNITS',
       u'LAND SQUARE FEET', u'GROSS SQUARE FEET', u'YEAR BUILT',
       u'TAX CLASS AT TIME OF SALE', u'BUILDING CLASS AT TIME OF SALE',
       u'SALE PRICE', u'SALE DATE', u'BOROUGHBLOCKLOT'],
      dtype='object')

In [7]:
# Changing header for easy merging
sales = sales.rename(columns={'BOROUGHBLOCKLOT': 'BBL'})
sales.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,BBL
0,1.0,ALPHABET CITY,03 THREE FAMILY DWELLINGS,1,376.0,24.0,,C0,264 EAST 7TH STREET,,...,0,3,"\t2,059","\t3,696",1900,1.0,C0,"\t7,738,700",12/22/16,137624.0
1,1.0,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,375.0,28.0,,C4,738 EAST 6TH STREET,,...,0,11,"\t1,750","\t6,500",1900,2.0,C4,"\t3,750,000",4/3/17,137528.0
2,1.0,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,385.0,36.0,,C7,27 AVENUE C,,...,1,25,"\t2,650","\t9,960",1910,2.0,C7,"\t5,235,000",7/11/17,138536.0
3,1.0,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392.0,5.0,,C2,151 AVENUE B,,...,0,5,"\t2,139","\t4,416",1900,2.0,C2,\t-,6/16/17,13925.0
4,1.0,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392.0,6.0,,C2,153 AVENUE B,,...,0,5,"\t1,633","\t6,440",1900,2.0,C2,"\t6,625,000",7/19/17,13926.0


In [8]:
# dropping nans and changing BBL to int for consistent formatting for merge
sales = sales[np.isfinite(sales['BBL'])]
sales.BBL = sales.BBL.astype(int)
sales.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,BBL
0,1.0,ALPHABET CITY,03 THREE FAMILY DWELLINGS,1,376.0,24.0,,C0,264 EAST 7TH STREET,,...,0,3,"\t2,059","\t3,696",1900,1.0,C0,"\t7,738,700",12/22/16,137624
1,1.0,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,375.0,28.0,,C4,738 EAST 6TH STREET,,...,0,11,"\t1,750","\t6,500",1900,2.0,C4,"\t3,750,000",4/3/17,137528
2,1.0,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,385.0,36.0,,C7,27 AVENUE C,,...,1,25,"\t2,650","\t9,960",1910,2.0,C7,"\t5,235,000",7/11/17,138536
3,1.0,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392.0,5.0,,C2,151 AVENUE B,,...,0,5,"\t2,139","\t4,416",1900,2.0,C2,\t-,6/16/17,13925
4,1.0,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392.0,6.0,,C2,153 AVENUE B,,...,0,5,"\t1,633","\t6,440",1900,2.0,C2,"\t6,625,000",7/19/17,13926


In [9]:
# force these to play nice
poi_pluto.BBL = sales.BBL

# merging
df = pd.merge(poi_pluto, sales, on='BBL')
df.head()


,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,MN,1440,49,108,118,5000,02,5,10065,L016,...,3,0,3,"\t2,059","\t3,696",1900,1.0,C0,"\t7,738,700",12/22/16
1,MN,1422,7501,108,118,3000,02,4,10065,L016,...,11,0,11,"\t1,750","\t6,500",1900,2.0,C4,"\t3,750,000",4/3/17
2,MN,1422,36,108,118,3000,02,4,10065,L016,...,24,1,25,"\t2,650","\t9,960",1910,2.0,C7,"\t5,235,000",7/11/17
3,MN,1422,2,108,118,3000,02,4,10065,L016,...,5,0,5,"\t2,139","\t4,416",1900,2.0,C2,\t-,6/16/17
4,MN,1424,43,108,126,4000,02,4,10021,L016,...,5,0,5,"\t1,633","\t6,440",1900,2.0,C2,"\t6,625,000",7/19/17


In [10]:
# saving to csv
df.to_csv('pluto_sales.csv')

___

### Finding time delta from sale to SAS opening 

In [11]:
# changing sale date to datetime
df['SALE DATE'] = pd.to_datetime(df['SALE DATE'])
# creating new column for subtraction with date of opening
df['sas'] = ('1/1/17')
# change to datetime
df['sas'] = pd.to_datetime(df['sas'])

In [12]:
# new column with change in time to sas opening
df['time'] = (df['SALE DATE'] - df['sas']).astype('timedelta64')
df.head()

,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,...,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,sas,time
0,MN,1440,49,108,118,5000,02,5,10065,L016,...,3,"\t2,059","\t3,696",1900,1.0,C0,"\t7,738,700",2016-12-22,2017-01-01,-8.640000e+14
1,MN,1422,7501,108,118,3000,02,4,10065,L016,...,11,"\t1,750","\t6,500",1900,2.0,C4,"\t3,750,000",2017-04-03,2017-01-01,7.948800e+15
2,MN,1422,36,108,118,3000,02,4,10065,L016,...,25,"\t2,650","\t9,960",1910,2.0,C7,"\t5,235,000",2017-07-11,2017-01-01,1.650240e+16
3,MN,1422,2,108,118,3000,02,4,10065,L016,...,5,"\t2,139","\t4,416",1900,2.0,C2,\t-,2017-06-16,2017-01-01,1.434240e+16
4,MN,1424,43,108,126,4000,02,4,10021,L016,...,5,"\t1,633","\t6,440",1900,2.0,C2,"\t6,625,000",2017-07-19,2017-01-01,1.719360e+16


In [13]:
# wtf, is this format correct? 
df.time.describe()

count    7.055000e+03
mean     7.434686e+15
std      8.808533e+15
min     -7.948800e+15
25%     -1.728000e+14
50%      7.430400e+15
75%      1.529280e+16
max      2.332800e+16
Name: time, dtype: float64

# To do
- plot average sales prices and time to sas opening
- build regression model with inputs predicting price: time to sas opening, distance to subway, (other housing characteristics)
- write the damn thing up
